In [1]:
import numpy as np
import pandas as pd
import json
import os 

import torch
import torch.nn as nn

In [2]:
path_and_labels = pd.read_csv('train.csv')

In [3]:
path_and_labels.head()

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie


In [4]:
print(path_and_labels['participant_id'].value_counts())

participant_id
49445    4968
61333    4900
36257    4896
16069    4848
26734    4841
55372    4826
2044     4810
37779    4782
32319    4753
29302    4722
22343    4677
53618    4656
37055    4648
28656    4563
62590    4563
34503    4545
27610    4275
25571    3865
18796    3502
4718     3499
30680    3338
Name: count, dtype: int64


Participant это скорее всего отдельные люди с которых собирались жесты. Для нас тут важно лишь то что данные не с одного человека и даже не с нескольких, тут данные с нескольких десятков участников, наша модель не будет учитывать особенности отдельных людей, более нас ничего в participant пока не интересует. 

In [5]:
k = 0
data = pd.read_parquet(path_and_labels.values[k, 0])
data.head()

,frame,row_id,type,landmark_index,x,y,z
0,20,20-face-0,face,0,0.494400,0.380470,-0.030626
1,20,20-face-1,face,1,0.496017,0.350735,-0.057565
2,20,20-face-2,face,2,0.500818,0.359343,-0.030283
3,20,20-face-3,face,3,0.489788,0.321780,-0.040622
4,20,20-face-4,face,4,0.495304,0.341821,-0.061152


In [6]:
len(data[data.frame == 20])

543

In [7]:
data[data.frame == 20].type.value_counts()

type
face          468
pose           33
left_hand      21
right_hand     21
Name: count, dtype: int64

In [8]:
data[data.frame == 20].type.value_counts()

type
face          468
pose           33
left_hand      21
right_hand     21
Name: count, dtype: int64

In [9]:
face_points = [0, 4, 6, 9, 13, 14, 17, 33, 37, 40, 46, 52, 55,
               61, 65, 78, 81, 82, 84, 87, 91, 95, 101, 102, 107,
               133, 145, 146, 148, 149, 159, 178, 185, 191, 195,
               199, 205, 263, 267, 270, 276, 282, 285, 291, 295,
               308, 311, 312, 314, 317, 321, 324, 330, 331, 336,
               362, 374, 375, 377, 378, 386, 402, 409, 415, 425]

print(face_points)
print()
points = [*face_points, *range(468, 543)]
print(points)
print(len(points))

[0, 4, 6, 9, 13, 14, 17, 33, 37, 40, 46, 52, 55, 61, 65, 78, 81, 82, 84, 87, 91, 95, 101, 102, 107, 133, 145, 146, 148, 149, 159, 178, 185, 191, 195, 199, 205, 263, 267, 270, 276, 282, 285, 291, 295, 308, 311, 312, 314, 317, 321, 324, 330, 331, 336, 362, 374, 375, 377, 378, 386, 402, 409, 415, 425]

[0, 4, 6, 9, 13, 14, 17, 33, 37, 40, 46, 52, 55, 61, 65, 78, 81, 82, 84, 87, 91, 95, 101, 102, 107, 133, 145, 146, 148, 149, 159, 178, 185, 191, 195, 199, 205, 263, 267, 270, 276, 282, 285, 291, 295, 308, 311, 312, 314, 317, 321, 324, 330, 331, 336, 362, 374, 375, 377, 378, 386, 402, 409, 415, 425, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542]
140


In [10]:
k = 0

data = pd.read_parquet(path_and_labels.values[k, 0])
A = data.loc[data.frame == data.frame.value_counts().index[0],["x", "y", "z"]]

A = A.to_numpy().reshape(1, 543, 3)
A = torch.tensor(A, dtype = torch.float32)
print(A.shape)

A = A[:, points]
print(A.shape)

A = A.reshape(1, -1)
print(A.shape)

torch.Size([1, 543, 3])
torch.Size([1, 140, 3])
torch.Size([1, 420])


In [11]:
k = 0

data = pd.read_parquet(path_and_labels.values[k, 0])
B = data.loc[:,["x", "y", "z"]]

B = B.to_numpy().reshape(len(data)//543, 543, 3)
B = torch.tensor(B, dtype=torch.float32)
print(B.shape)

B = B[:, points]
print(B.shape)

B = B.reshape(B.shape[0], -1)
print(B.shape)

torch.Size([23, 543, 3])
torch.Size([23, 140, 3])
torch.Size([23, 420])


In [12]:
train_data = path_and_labels.copy()

In [13]:
partic_list = path_and_labels['participant_id'].value_counts().index
for partic in partic_list:
    
    directory = "train_data/" + str(partic)
    if not os.path.exists(directory):
        os.makedirs(directory)
        print("make:", directory)


make: train_data/49445
make: train_data/61333
make: train_data/36257
make: train_data/16069
make: train_data/26734
make: train_data/55372
make: train_data/2044
make: train_data/37779
make: train_data/32319
make: train_data/29302
make: train_data/22343
make: train_data/53618
make: train_data/37055
make: train_data/28656
make: train_data/62590
make: train_data/34503
make: train_data/27610
make: train_data/25571
make: train_data/18796
make: train_data/4718
make: train_data/30680


In [14]:
for i in range(len(path_and_labels)):
    path = path_and_labels.values[i, 0]
    
    data = pd.read_parquet(path)
    data = data.loc[:,["x", "y", "z"]]    
    data = data.to_numpy().reshape(len(data)//543, 543, 3)[:, points]
    data = data.reshape(data.shape[0], -1)
    np.nan_to_num(data, nan=0)
    
    path = path.split("/")
    path[0] = "train_data"
    path[-1] = path[-1][:-8]
    path = "/".join(path) 
    path = path + '.npy'
    
    train_data.iloc[i, 0] = path
    
    np.save(path, data)
    
    if i % 1000 == 0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000


In [15]:
train_data

,path,participant_id,sequence_id,sign
0,train_data/26734/1000035562.npy,26734,1000035562,blow
1,train_data/28656/1000106739.npy,28656,1000106739,wait
2,train_data/16069/100015657.npy,16069,100015657,cloud
3,train_data/25571/1000210073.npy,25571,1000210073,bird
4,train_data/62590/1000240708.npy,62590,1000240708,owie
...,...,...,...,...
94472,train_data/53618/999786174.npy,53618,999786174,white
94473,train_data/26734/999799849.npy,26734,999799849,have
94474,train_data/25571/999833418.npy,25571,999833418,flower
94475,train_data/29302/999895257.npy,29302,999895257,room


In [19]:
train_data.to_csv("train_table.csv", index=False)